In [5]:
import cv2
import numpy as np
import face_recognition
import os
import torch
import time

In [20]:
# def load_yolo(model_path, conf_thres=0.5, iou_thres=0.4):
#     model = cv2.dnn.readNet(model_path)
#     return model, conf_thres, iou_thres

def load_yolo(model_path, conf_thres=0.5, iou_thres=0.4):
    # Load YOLOv5 model from PyTorch Hub
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
    model.conf = conf_thres  # Set confidence threshold
    model.iou = iou_thres    # Set IoU threshold
    return model

def register_faces():
    known_face_encodings = []
    known_face_names = []

    # Dictionary mapping desks to folder paths containing images of the respective desk owners
    owner_image_folders = {
        "desk1": "Downloads/converted_keras/self data/",  # Folder containing images of owner A
        "desk2": "Downloads/desk2/",  # Folder containing images of owner B
    }

    for desk, folder_path in owner_image_folders.items():
        if os.path.exists(folder_path):
            # Loop through all image files in the folder
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                # Load the image and compute face encodings
                owner_image = face_recognition.load_image_file(image_path)
                owner_encodings = face_recognition.face_encodings(owner_image)
                if owner_encodings:
                    known_face_encodings.append(owner_encodings[0])
                    known_face_names.append(desk)

    return known_face_encodings, known_face_names

def zone_boundary():
    desk_boundaries = {
        "desk1": (50, 20, 300, 480),  # (x1, y1, x2, y2)
        "desk2": (350, 20, 600, 480),
    }
    return desk_boundaries

def draw_boundary_boxes(frame, desk_boundaries):
    for desk, (x1, y1, x2, y2) in desk_boundaries.items():
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, desk, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def intersection_area(rect1, rect2):
    x1 = max(rect1[0], rect2[0])
    y1 = max(rect1[1], rect2[1])
    x2 = min(rect1[0] + rect1[2], rect2[0] + rect2[2])
    y2 = min(rect1[1] + rect1[3], rect2[1] + rect2[3])
    if x1 < x2 and y1 < y2:
        return (x2 - x1) * (y2 - y1)
    return 0

def is_outside_boundary(box, boundary):
    person_area = box[2] * box[3]
    intersection = intersection_area(box, boundary)
    inside_ratio = intersection / person_area
    return inside_ratio < 0.3  # Less than 30% inside means more than 70% outside

def process_frame(frame, model, desk_boundaries, last_seen, alert_displayed, known_face_encodings, known_face_names):
    # YOLOv5 inference
    results = model(frame)

    # Convert YOLOv5 results to a format compatible with the rest of your code
    boxes = []
    confidences = []
    class_ids = []

    for *box, conf, cls in results.xyxy[0].cpu().numpy():
        x1, y1, x2, y2 = map(int, box)
        w = x2 - x1
        h = y2 - y1
        class_id = int(cls)
        confidence = conf

        if class_id == 0:  # Only consider 'person' class
            boxes.append([x1, y1, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, model.conf, model.iou)

    # Initialize person_in_desk outside the conditional block
    person_in_desk = {desk: False for desk in desk_boundaries.keys()}

    # Check if indexes is not empty
    if len(indexes) > 0:
        indexes = indexes.flatten()

        for i in indexes:
            x, y, w, h = boxes[i]
            person_bbox = (x, y, w, h)

            # Perform facial recognition
            face_locations = face_recognition.face_locations(frame)
            face_encodings = face_recognition.face_encodings(frame, face_locations)

            for face_encoding, face_location in zip(face_encodings, face_locations):
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)

                if matches[best_match_index]:
                    desk = known_face_names[best_match_index]
                    if not is_outside_boundary(person_bbox, (desk_boundaries[desk][0], desk_boundaries[desk][1], desk_boundaries[desk][2] - desk_boundaries[desk][0], desk_boundaries[desk][3] - desk_boundaries[desk][1])):
                        person_in_desk[desk] = True
                        last_seen[desk] = time.time()
                        alert_displayed[desk] = False
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                        cv2.putText(frame, f"{desk} - Owner", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                    else:
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                        cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Check alert conditions for each desk
    for desk in desk_boundaries.keys():
        if not person_in_desk[desk]:
            if time.time() - last_seen[desk] > 5:  # 60 seconds = 1 minute
                alert_displayed[desk] = True
        if alert_displayed[desk]:
            cv2.putText(frame, f"Alert: {desk} left for over 1 minute!", (50, 50 + 30 * list(desk_boundaries.keys()).index(desk)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

def main():
    model_path = input("Enter the path for the YOLO model file: ")

    # Load the model without unpacking
    model = load_yolo(model_path)

    desk_boundaries = zone_boundary()

    known_face_encodings, known_face_names = register_faces()

    last_seen = {desk: time.time() for desk in desk_boundaries.keys()}
    alert_displayed = {desk: False for desk in desk_boundaries.keys()}

    cap = cv2.VideoCapture(0)  # Change 0 to your camera source

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        draw_boundary_boxes(frame, desk_boundaries)
        process_frame(frame, model, desk_boundaries, last_seen, alert_displayed, known_face_encodings, known_face_names)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



In [21]:
if __name__ == "__main__":
    main()


Enter the path for the YOLO model file:  Downloads/yolov5s.pt


Using cache found in C:\Users\champ/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-8-5 Python-3.12.4 torch-2.4.0 CPU

Fusing layers... 
Model summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
C:\Users\champ/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\champ/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\champ/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\champ/.cache\torch\hub\ultralytics_yolov5_master\models\common.p

In [16]:
cap = cv2.VideoCapture(0)

In [17]:
cap.release()
cv2.destroyAllWindows()